# Computo de alto desempeño
## Elízabeth Daniela Merchán Cardoza

#### Semana 2 - Mayo 10
Importar liberias necesarias y verificar versiones
#### Objetivo: 
Configurar el software que se usarádurante el curso para desarrollar aplicaciones de computo paralelo

In [1]:
import numpy as np
print np.__version__

import pandas as pd
print pd.__version__

import matplotlib as plt
print plt.__version__

import ipyparallel as ipp
print ipp.__version__

1.11.3
0.19.2
2.0.0
6.0.2


##### Modulo ipypararrel instalado correctamente
Luego de importar el modulo se crea la instancia _Cliente_:
Se deben ingresar primero los motores con los que se ejecutará la instancia

In [2]:
c = ipp.Client()

## Motores conectados al controlador
Se ejecuta la siguiente instrucción para ver la lista de ids de los motores disponibles. Debe coincidir con el numero especificado anteriormente.

In [3]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [4]:
c

Esta forma supone que la coneccion de facto es correcta. Dicha informacion puede verse en $IPYTHONDIR/profile_default/security_$, así:

In [5]:
!dir C:\Users\AlbertoC\.ipython\profile_default\security

 El volumen de la unidad C no tiene etiqueta.
 El n£mero de serie del volumen es: D4F5-8218

 Directorio de C:\Users\AlbertoC\.ipython\profile_default\security

15/05/2017  04:28 p.m.    <DIR>          .
15/05/2017  04:28 p.m.    <DIR>          ..
15/05/2017  04:28 p.m.               382 ipcontroller-client.json
15/05/2017  04:28 p.m.               367 ipcontroller-engine.json
               2 archivos            749 bytes
               2 dirs  310.550.310.912 bytes libres


Se verifcica la información del archivo:

In [6]:
# %load C:\Users\AlbertoC\.ipython\profile_default\security\ipcontroller-client.json
{
  "control": 59373, 
  "task": 59379, 
  "notification": 59383, 
  "task_scheme": "leastload", 
  "mux": 59375, 
  "iopub": 59381, 
  "ssh": "", 
  "key": "2882a98e-89b1-41d2-8012-c841999823b6", 
  "registration": 59367, 
  "interface": "tcp://127.0.0.1", 
  "signature_scheme": "hmac-sha256", 
  "pack": "json", 
  "unpack": "json", 
  "location": "192.168.0.13"
}

{'control': 59373,
 'interface': 'tcp://127.0.0.1',
 'iopub': 59381,
 'key': '2882a98e-89b1-41d2-8012-c841999823b6',
 'location': '192.168.0.13',
 'mux': 59375,
 'notification': 59383,
 'pack': 'json',
 'registration': 59367,
 'signature_scheme': 'hmac-sha256',
 'ssh': '',
 'task': 59379,
 'task_scheme': 'leastload',
 'unpack': 'json'}

## Ejecucion directa
Para la ejecucion directa usamos un objeto DirectView que puede construirse a partir de la lista de acceso al cliente:

In [7]:
c[:]

<DirectView [0, 1, 2, 3,...]>

In [8]:
"Hola alumnos"

'Hola alumnos'

In [11]:
# utiliza solo un motor
c[0].apply_sync(lambda : "Hola alumnos") 

'Hola alumnos'

In [12]:
# utiliza todos los motores que se definieron inicialmente
c[:].apply_sync(lambda : "Hola alumnos") 

['Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos']

In [13]:
#se asignan todos los motores a una varible
dview = c[:]
dview.apply_sync(lambda : "Hola alumnos")

['Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos',
 'Hola alumnos']

In [14]:
#debe mostrar la lista de todos los motores
dview 

<DirectView [0, 1, 2, 3,...]>

In [18]:
#equivalente a dview = c[:] 
dview = c.direct_view()

Id de procesos de cada motor

In [19]:
#modulo del sistema operativo
import os 
dview.apply_sync(os.getpid)

[5076, 10408, 8132, 8492, 6424, 10544, 4232, 9464]

In [20]:
import socket
dview.apply_sync(socket.gethostname)

['DESKTOP-06SSJNP',
 'DESKTOP-06SSJNP',
 'DESKTOP-06SSJNP',
 'DESKTOP-06SSJNP',
 'DESKTOP-06SSJNP',
 'DESKTOP-06SSJNP',
 'DESKTOP-06SSJNP',
 'DESKTOP-06SSJNP']

## Direct view
* Una vista proporciona acceso a un subconjunto de motores disponibles para el cliente
* Las tareas se asignan a los motores usando la vista
* DirectView permite enviar tareas de manera explicita a un motor especifico

### Ejercicio con direct view
Sumemos 10 conjuntos de 3 numeros en paralelo usando todos los motores en la vista

In [24]:
dview.map_sync(lambda x, y, z: x+y+z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Ahora sumemos 10 conjuntos de 3 numeros en paralelo usando motores alternados

In [25]:
#usa motores alternando de 2 en 2
c[::2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Sumemos 10 conjuntos de 3 numeros usando un motor especifico: 3

In [26]:
c[3].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

## Load balanced view
* LoadBalancedView administra la distribucion de tareas y es útil cuando se tienen muchas tareas que toman diferentes tiempos para ejecutarse

Se importa numpy en caso de no haberlo hecho:
<hb>_import numy as np_


In [27]:
lview = c.load_balanced_view()

In [28]:
lview.map_sync(lambda x: sum(x), np.random.random((10,1000000)))

[500379.54873063433,
 500265.9273859883,
 499621.99576848734,
 499992.52688407147,
 500462.73619443586,
 500036.53806735872,
 500099.71843747899,
 500048.75071417756,
 499789.927545354,
 500312.10932609549]

## Map
Python tiene una funcion interna llamada map que permite aplicar cualquier funcion lemento a elemente a una secuencia. Para este ejemplo utilizaremos una funcion muy sencilla definida a continuacion.

In [30]:
def cubo(x):
    return x**3
cubo(2)

8

In [33]:
lista1 = [cubo(x) for x in range (1,7)]
lista1

[1, 8, 27, 64, 125, 216]

In [35]:
map(cubo, range(1,7))

[1, 8, 27, 64, 125, 216]

In [38]:
serial = map(cubo, range(1,70))
serial

[1,
 8,
 27,
 64,
 125,
 216,
 343,
 512,
 729,
 1000,
 1331,
 1728,
 2197,
 2744,
 3375,
 4096,
 4913,
 5832,
 6859,
 8000,
 9261,
 10648,
 12167,
 13824,
 15625,
 17576,
 19683,
 21952,
 24389,
 27000,
 29791,
 32768,
 35937,
 39304,
 42875,
 46656,
 50653,
 54872,
 59319,
 64000,
 68921,
 74088,
 79507,
 85184,
 91125,
 97336,
 103823,
 110592,
 117649,
 125000,
 132651,
 140608,
 148877,
 157464,
 166375,
 175616,
 185193,
 195112,
 205379,
 216000,
 226981,
 238328,
 250047,
 262144,
 274625,
 287496,
 300763,
 314432,
 328509]

In [39]:
parallel = dview.map_sync(cube, range(1,70))

In [40]:
serial == parallel

True

In [41]:
%timeit map(cube, range(1,70))
%timeit dview.map_sync(cube, range(1,70))

100000 loops, best of 3: 12.3 µs per loop
10 loops, best of 3: 96.2 ms per loop


In [42]:
dview.map_sync(cube, range(1,7))

[1, 8, 27, 64, 125, 216]

In [43]:
def multi(a,b):
    return a*b
dview.map_sync(multi, [0,1,3,5],[2,4,6,8])

[0, 4, 18, 40]

In [44]:
PrimeQ = lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1

In [46]:
PrimeQ(2)

True

In [47]:
PrimeQ(28)

False

In [49]:
PrimeQ(23)

True